In [5]:
# Package that provides access to Fits files 
from astropy.io import fits     
import numpy
import OpenEXR

In [ ]:
def fits_to_exr(input_fits, output_exr32):
    
    # Returns HDUList object which is a list-like collection of HDU objects.
    # Each HDU contains a header and data. Header contains keyword, value, and comment. Keyword / value
    # can be manipulated like a python dict. Data is a numpy ndarray (n-dimensional array). 
    # For now, we only look at the first HDU (this should be changed later). 
    HDUList = fits.open(input_fits)
    fits_header = HDUList[0].header 
    fits_data = HDUList[0].data
    HDUList.close()
    
    # Get the dimension of the image. 
    width, height = fits_data.shape
    
    # Convert the elements in fitz_data to float32 format (presumably from float64, but this should be checked).
    # This can be done analogously for 16 bit. 
    fits_data_f32 = numpy.float32(fits_data)
    
    # Create the EXR header with appropriate height and width.
    # Copy the Fits header to EXR. Probably need to do more of a deep copy later. 
    header = OpenEXR.Header(height, width)
    header['header'] = fits_header
    
    # Create EXR output file with header. Write Fits data to EXR red channel. 
    EXR32 = OpenEXR.OutputFile(output_exr32, header)
    EXR32.writePixels({"R": fits_data_f32})
    
    return output_exr32
    

In [17]:
fits_input = "/home/idies/workspace/Storage/mtaghiza/exr/SkyServer FITS/2MASS J11582332-0020114/frame-r-006793-3-0127.fits"
exr_output = "exr_demo.exr"
fits_to_exr(fits_input, exr_output)

'exr_demo.exr'